##### Appendix B: Problem 6

Given information - Components names, Molecular Weights & Molar Fractions

In [23]:
# From table B-9
names = ['C1', 'C2', 'C3', 'iC4', 'nC4', 'iC5', 'nC5', 'C6', 'C7+']
y = [0.875, 0.083, 0.021, 0.006, 0.008, 0.003, 0.002, 0.001, 0.001]
R = 10.73146 # Universal gas constant, psia-ft3/R-lbm mol

# From Table 10
z = y
M = [16.04, 30.07, 44.09, 58.12, 58.12, 72.15, 72.15, 86.17, 114.0]
pc = [667.8, 707.8, 616.3, 529.1, 550.7, 490.4, 488.6, 436.9, 360.6]
tc = [343, 549.8, 665.7, 734.7, 765.3, 828.3, 845.4, 913.4, 1023.9]

In [24]:
# Solution
ncomps = len(y)
zM = [z[i]*M[i] for i in range(ncomps)]
zpc = [z[i]*pc[i] for i in range(ncomps)]
ztc = [z[i]*tc[i] for i in range(ncomps)]
Mg = sum(zM)
Tpc = sum(ztc)
Ppc = sum(zpc)
sg = Mg/28.97
t = 160 # deg F
Tpr = (t + 460)/Tpc
p = 2000+14.7
Ppr = p/Ppc

Programming up the Hall Yarborough Z-factor equation, leveraging the Python fsolve solving algorithm.

In [25]:
# Hall-Yarborough equation - Section 3.3.2
import math
from scipy.optimize import fsolve

def fy(y, *args):
    alpha, Pr, t = args
    return -alpha * Pr + (y + y ** 2 + y ** 3 - y ** 4) / (1 - y) ** 3 - (14.76 * t - 9.76 * t ** 2 + 4.58 * t ** 3) * y ** 2 + (90.7 * t - 242.2 * t ** 2 + 42.4 * t ** 3) * y ** (2.18 + 2.82 * t)
    
def Zfac(Tr, Pr):
    t = 1 / Tr
    alpha = 0.06125 * t * math.exp(-1.2 * (1 - t) ** 2)
    args = (alpha, Pr, t)
    y = 0.001
    y = fsolve(fy, y, args=args)
    return alpha * Pr / y

In [26]:
Z = Zfac(Tpr, Ppr)[0]
rho_g = (p*Mg)/(Z*R*(t+460))
one_on_bg = 1 / (Z * (t + 460) / (p * 35.37))

In [27]:
Tpchc = 169.2 + 349.5*sg - 74*sg**2
Ppchc = 756.8 - 131*sg - 3.6 * sg**2
Tprhc = (t+460)/Tpchc
Pprhc = p/Ppchc
Zhc = Zfac(Tprhc, Pprhc)[0]
rho_ghc = (p*Mg)/(Zhc*R*(t+460))
one_on_bg_hc = 1 / (Zhc * (t + 460) / (p * 35.37))

In [28]:
#Print out results
from tabulate import tabulate
print('Problem 6 Summary Results'+'\n')
header=['Method', 'Gas MW', 'sg','Tpc', 'Ppc',  'Tpr', 'Ppr', 'Z', '1/Bg', 'Rhog']
result_table = [['From Composition', Mg, round(sg,4),Tpc, Ppc,  Tpr, Ppr, Z, one_on_bg, rho_g]]
result_table.append(['From SG Correlations', ' ', ' ', Tpchc, Ppchc,  Tprhc, Pprhc, Zhc, one_on_bg_hc, rho_ghc])
print(tabulate(result_table,headers=header))

Problem 6 Summary Results

Method                Gas MW    sg        Tpc      Ppc      Tpr      Ppr         Z     1/Bg     Rhog
--------------------  --------  ----  -------  -------  -------  -------  --------  -------  -------
From Composition      18.8313   0.65  376.382  666.841  1.64726  3.02126  0.845176  135.99   6.74673
From SG Correlations                  365.117  670.125  1.69809  3.00645  0.86427   132.986  6.59767
